# Redes Sociais

## Estudando uma rede de coautoria de artigos científicos 

### GRUPO-L
- Anderson Franco
- Guilherme Paraíso
- Vinícius Rodrigues

# Dataset

Nossa base de dados se trata de artigos da aréa de Medicina entre os anos de 2000 e 2005

[https://drive.google.com/file/d/1WZP0l9qdJOUkwsZhwrYW7U_8ytbob0WA/view?usp=sharing](https://drive.google.com/file/d/1WZP0l9qdJOUkwsZhwrYW7U_8ytbob0WA/view?usp=sharing)

| Nome | Datasets usados para construir a rede | Tipo de Rede | Operacionalização dos vértices | Operacionalização das arestas  |
|---|---|---|---|---|
| Authorship network | Scopus Articles | One-mode comum | Os vértices da rede representam autores de artigos científicos. O dataset foi processado linha por linha, onde cada linha tem informações sobre um artigo como título e nomes dos autores. Os autores de cada artigo foram extraídos e armazenados e em seguida, cada autor único foi adicionado ao grafo como um vértice usando o método g.add_vertex_by_id(autor). Dessa forma, cada vértice no grafo corresponde a um autor presente no dataset. | As arestas da rede representam coautorias entre autores, indicando que dois autores escreveram pelo menos um artigo juntos. Foram identificados todos os autores de cada artigo. Para cada par de coautores, uma aresta foi adicionada ao grafo usando o método g.add_edge_by_ids(autor1, autor2). Como o grafo é não direcionado, cada aresta é única, evitando duplicações. Assim, as arestas capturam as relações de colaboração entre os autores. |

Agora vamos construir a estrutura da rede para visualização

In [ ]:
import graph_tool_extras as gte
import pandas as pd
import csv

In [ ]:
PATH = '../../Datasets/scopus (14).csv'

In [ ]:
g = gte.Graph(directed=False)

In [ ]:
artigos_para_autores = {}

# Lê o arquivo CSV
with open(PATH) as file:
    reader = csv.reader(file)
    next(reader)  # Ignora o cabeçalho

    for line in reader:
        autores = line[0].split(';')  # Lista de autores
        titulo = line[3]  # Título do artigo

        # Adiciona o artigo ao dicionário de artigos para autores
        artigos_para_autores[titulo] = autores

# Adiciona todos os autores como vértices no grafo
autores_set = set()  # Usamos um conjunto para evitar duplicatas
for autores in artigos_para_autores.values():
    autores_set.update(autores)

for autor in autores_set:
    g.add_vertex_by_id(autor)

for autores in artigos_para_autores.values():
    # Itera sobre todos os pares únicos de autores no mesmo artigo
    for i in range(len(autores)):
        for j in range(i + 1, len(autores)):
            g.add_edge_by_ids(autores[i], autores[j])

Aqui abaixo vamos gerar o grafo da nossa rede, e fazer a visualização

In [ ]:
from graph_tool import draw

In [ ]:
layout = draw.sfdp_layout(g)

In [ ]:
gte.move(g, layout)

In [ ]:
g = gte.clean(g) 

In [ ]:
gte.save(g, 'artigo.net.gz')

In [ ]:
import netpixi

In [ ]:
resultado = netpixi.render('artigo.net.gz')

In [ ]:
resultado.vertex_default(
    size=4,         # tamanho            (padrão: 16)
    bwidth=2,        # espessura da borda (padrão: 2)
)

In [ ]:
resultado.edge_default(
    width=0.6,        # espessura         (padrão: 2)
)

# Visualização da rede

![](artigo.net.gz1.png)

# Visualização do Degree da rede

## O tamanho dos vértices corresponde ao degree

In [ ]:
#DEGREE
g = gte.load('artigo.net.gz')

c = gte.total_degree(g)

g.add_vp('degree', c)

In [ ]:
gte.save(g, 'degree.net.gz')

In [ ]:
r = netpixi.render('degree.net.gz')

In [ ]:
r.vertex_scale('degree', 10, 30)

# Visualização do Betweenness da rede

## O tamanho dos vértices corresponde ao betweenness

In [ ]:
#BETWEENEES
from graph_tool import centrality
g = gte.load('artigo.net.gz')

c, _ = centrality.betweenness(g)

g.add_vp('betweenness', c)

In [ ]:
gte.save(g, 'betweenness.net.gz')

In [ ]:
r = netpixi.render('betweenness.net.gz')

In [ ]:
r.vertex_scale('betweenness', 4, 30)

# Visualização do closeness da rede

## O tamanho dos vértices corresponde ao closeness


In [ ]:
#CLOSENESS
from graph_tool import centrality
g = gte.load('artigo.net.gz')

c = centrality.closeness(g, harmonic=True)

g.add_vp('closeness', c)

In [ ]:
gte.save(g, 'closeness.net.gz')

In [ ]:
r = netpixi.render('closeness.net.gz')

In [ ]:
r.vertex_scale('closeness', 4, 20)

# Visualização da restrição da rede

## O tamanho dos vértices corresponde a restrição

In [ ]:
#RESTRICAO
from graph_tool import centrality
g = gte.load('artigo.net.gz')

c = gte.burt_constraint(g)

g.add_vp('burt_constraint', c)

In [ ]:
gte.save(g, 'structural_holes.net.gz')

In [ ]:
r = netpixi.render('structural_holes.net.gz')

In [ ]:
r.vertex_scale('burt_constraint', 4, 30)

# Visualização do coreness continuo da rede

## O tamanho dos vértices corresponde ao coreness continuo

In [ ]:
#CONTINUO
import cpnet
import netpixi
g = gte.load('artigo.net.gz')

c, l = gte.coreness(g, cpnet.Rombach)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.hist(c.a, bins=50, edgecolor='black', alpha=0.7) # 'c.a' acessa o array numpy dos valores de coreness
plt.title('Distribuição dos Valores de Coreness')
plt.xlabel('Valor de Coreness')
plt.ylabel('Frequência de Nós')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
g.add_vp('coreness', c)

gte.save(g, 'coreness.net.gz')

r = netpixi.render('coreness.net.gz')

In [ ]:
r.vertex_scale('coreness', 5, 20)

# Visualização do coreness discreto da rede

## O tamanho dos vértices corresponde ao coreness discreto

In [ ]:
#DISCRETO
import cpnet
import netpixi

g = gte.load('artigo.net.gz')

c, l = gte.coreness(g, cpnet.Surprise)

g.add_converted_vp('color', c, lambda coreness: 0x00ff00 if round(coreness) == 1 else 0xff0000)

gte.save(g, 'coreness_discreto.net.gz')

r = netpixi.render('coreness_discreto.net.gz');

In [ ]:
r.vertex_scale('coreness', 5, 50)

# Distriguição do Degree

In [ ]:
#DISTRIBUICAO DEGREE
g = gte.load('artigo.net.gz')
degrees = g.get_total_degrees()

In [ ]:
degrees.describe()

In [ ]:
degrees.hist();

# Construção do Mecanismo

A formulação de um "mecanismo" em análise de redes sociais é um processo estruturado que visa explicar como uma variável independente (geralmente uma métrica de rede) influencia uma variável dependente, através de ações e entidades específicas, e sob certas condições. Ele vai além de uma simples correlação, buscando a causalidade e a explicação dos processos.


## 1. Definir Variáveis (Independente e Dependente):

    - Variável Independente: Escolhe uma métrica de rede (e.g., Betweenness centrality, Degree, Closeness centrality, etc.) que se acredita ter um papel causal. Esta é a característica da rede que será investigada como influência.
    - Variável Dependente: Identifica o fenômeno ou resultado que se quer explicar (e.g., coautoria internacional, inovação, transferência de conhecimento). Esta é a variável de resultado.

## 2. Formular a Hipótese:

Com base nas variáveis, cria-se uma frase clara e concisa que articula a relação esperada entre elas. Pensa-se em termos de "o aumento de X leva a Y" ou "maior X está associado a maior Y". Esta é a afirmação testável.

## 3. Identificar as Entidades do Mecanismo:

Pergunta-se: "Quem age neste mecanismo?" ou "Quais são os agentes principais envolvidos no processo?" São os nós da rede que estão diretamente envolvidos na explicação.

## 4. Descrever as Ações do Mecanismo:

Aqui, detalha-se o que as entidades identificadas fazem que leva ao resultado. Aqui é importante pensar nos comportamentos, interações ou processos que transformam a variável independente na variável dependente. "Como" a entidade influencia o resultado?

## 5. Especificar os Resultados do Mecanismo:

Qual é o resultado final gerado pelas ações das entidades? Deve ser um resultado observável e, idealmente, mensurável, que se alinha com a variável dependente.

## 6. Definir o Contexto do Mecanismo:

Nenhum mecanismo ocorre no vácuo. Quais são as condições explícitas ou implícitas (sociais, organizacionais, políticas, temporais) que precisam estar presentes para que o mecanismo funcione como esperado? Quais fatores poderiam fortalecer ou enfraquecer o efeito?

## 7. Redação Final do Mecanismo:

Reúnem-se todos os elementos anteriores em um texto coeso e explicativo. Esta é a narrativa detalhada do mecanismo. Começa-se com a variável independente e segue-se a lógica através das entidades, ações, resultados e contexto. Usa-se uma linguagem clara e evita-se jargões desnecessários, explicando a lógica subjacente.

Ao seguir esses passos, constrói-se uma explicação robusta e testável de como as estruturas de rede influenciam resultados específicos, o que é fundamental para a compreensão aprofundada em análise de redes.


# Nosso Mecanismo

| | |
|---|---|
| Variável independente  | Betweenness centrality |
| Variável dependente | coautoria internacional |
| Hipótese | Autores com alta betweenness centrality têm maior probabilidade de publicar em coautoria com autores de diferentes países. |
| Entidades do mecanismo (quem age?) | autores |
| Ações do mecanismo (o que as entidades fazem? o que gera algo?) | Esses autores mediam colaborações internacionais estratégicas negociando projetos que dependem de recursos geograficamente dispersos, viabilizando parcerias que outros não conseguiriam. |
| Resultados do mecanismo (qual é o resultado final gerado?) | Maior probabilidade de artigos com coautoria internacional |
| Contexto do mecanismo (quais são as condições explícitas ou implícitas que, se fossem alteradas, modificariam os resultados?) | O efeito é mais forte em áreas com especialização regional ou em países com políticas de fomento à colaboração internacional, mas enfraquece em redes já globalizadas ou sob restrições políticas. |
| Redação final do mecanismo | Autores com alta betweenness centrality atuam como pontes entre comunidades científicas separadas, muitas vezes com diferentes especializações, localizações geográficas ou idiomas. Diferente de autores com alto degree (que têm muitos coautores diretos), esses autores circulam entre subgrupos desconectados, e com isso frequentemente estabelecem conexões que outros não fariam. Esse posicionamento estrutural permite que eles cruzem barreiras institucionais, disciplinares e geográficas, abrindo portas para colaborações internacionais estratégicas, mesmo sem ter grande número de conexões totais. Muitas vezes, essas colaborações são funcionais, motivadas por complementaridade metodológica, acesso a dados únicos, ou por estarem em posição de negociar e intermediar interesses entre grupos distintos. |

# Tratando os dados

Agora vamos tratar os dados para trabalhar com a nossa hipótese de coautoria internacional. Nosso dataset contém as seguintes colunas:

- "Authors"
- Authors full names
- Authors IDs
- Title
- Year
- Cited by
- Link
- Affiliations
- Authors with affiliations

Iremos trabalhar com a coluna de Affiliations para obter as informações de coautoria internacional

In [ ]:
import pandas as pd
import csv
import graph_tool.all as gt # Usaremos graph_tool para as métricas de rede
import numpy as np # Importar numpy para log e NaN
import statsmodels.api as sm # Para OLS
import statsmodels.formula.api as smf # Para OLS com fórmula
from statsmodels.discrete.discrete_model import Logit # Para Regressão Logística
import re # Para extrair países

PATH = '../../Datasets/scopus (14).csv'

# --- Construção do Grafo e Cálculo das Métricas de Autor ---

# Criar grafo vazio
g = gt.Graph(directed=False)

# Dicionário para mapear autores para vértices
autor_para_vertice = {}
# Dicionário para armazenar o nome do autor associado a cada vértice
vertice_para_autor = {}

print("Construindo o grafo a partir do CSV...")

author_col_idx = 0
affiliation_col_idx = -1 # Inicializamos affiliation_col_idx para ser encontrado pelo nome

with open(PATH, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)  # Pular cabeçalho

    # Encontrar o índice da coluna de afiliações pelo nome
    try:
        affiliation_col_idx = header.index('Affiliations')
    except ValueError as e:
        print(f"Erro: Coluna 'Affiliations' não encontrada no cabeçalho. Verifique o nome da coluna. {e}")
        # Se a coluna de afiliações não for encontrada, não podemos continuar
        exit()

    # Verificar se o índice de afiliações foi encontrado com sucesso
    if affiliation_col_idx == -1:
        print("Erro: A coluna 'Affiliations' não foi encontrada.")
        exit()

    # Não precisamos verificar author_col_idx aqui, pois assumimos que é 0

    for i, line in enumerate(reader):
        # Ignorar linhas com dados insuficientes ou erros, usando os índices encontrados
        # Verificamos se o comprimento da linha é suficiente para acessar o índice de afiliações
        # e assumimos que o índice 0 (autores) sempre existe se houver afiliações ou outros dados
        if len(line) <= affiliation_col_idx:
             continue

        # Acessar a coluna de autores diretamente pelo índice 0
        autores_str = line[author_col_idx]
        if not isinstance(autores_str, str) or autores_str.strip() == '':
            continue # Pula linhas sem autores

        autores = [a.strip() for a in autores_str.split(';') if a.strip()] # Garante que não há autores vazios

        if len(autores) < 1:
            continue # Pula artigos sem autores válidos

        # Adicionar autores ao grafo se não existirem
        for autor in autores:
            if autor not in autor_para_vertice:
                v = g.add_vertex()
                autor_para_vertice[autor] = v
                vertice_para_autor[g.vertex_index[v]] = autor # Mapeia índice do vértice para nome do autor

        # Adicionar arestas entre coautores no mesmo artigo
        for i_autor in range(len(autores)):
            for j_autor in range(i_autor + 1, len(autores)):
                autor1 = autores[i_autor]
                autor2 = autores[j_autor]
                v1 = autor_para_vertice[autor1]
                v2 = autor_para_vertice[autor2]
                # Adiciona aresta apenas se não existir para evitar duplicatas em grafos não direcionados
                if g.edge(v1, v2) is None:
                    g.add_edge(v1, v2)

In [ ]:
# Calcular o degree de cada autor
print("Calculando degree dos autores...")
degree_vp = g.degree_property_map("total")
autor_degree = {vertice_para_autor[g.vertex_index[v]]: degree_vp[v] for v in g.vertices()}

# Calcular a centralidade de intermediação de cada autor
print("Calculando betweenness centrality dos autores...")
# betweenness() retorna uma tupla de propriedades de vértice: betweenness e edge_betweenness
vb, eb = gt.betweenness(g)
autor_betweenness = {vertice_para_autor[g.vertex_index[v]]: vb[v] for v in g.vertices()}

In [ ]:
print("Carregando e preparando o DataFrame...")
df = pd.read_csv(PATH)
df['Authors'] = df['Authors'].fillna('')  # Substitui NaN por string vazia
df['Affiliations'] = df['Affiliations'].fillna('') # Substitui NaN por string vazia

# Função para extrair países (reutilizada da sua versão corrigida)
def extract_countries(affiliations_str):
    if not isinstance(affiliations_str, str) or affiliations_str.strip() == '':
        return []

    countries = []
    # Divide a string em afiliações individuais usando ponto e vírgula
    affiliations_list = affiliations_str.split(';')

    for affiliation in affiliations_list:
        # Divide cada afiliação por vírgulas
        parts = [p.strip() for p in affiliation.split(',')]
        # Pega o último elemento, que esperamos que seja o país
        if parts: # Garante que a lista de partes não está vazia
            country = parts[-1].strip()
            if country and len(country) > 1: # Garante que o nome do país não está vazio e tem mais de 1 caractere
                 countries.append(country)

    # Remove duplicatas e retorna como lista
    return list(set(countries))

# Aplicar a função para extrair países
df['countries'] = df['Affiliations'].apply(extract_countries)

# Calcular o número de países e a variável dependente logaritmizada
df['num_paises'] = df['countries'].apply(len)

df_nm = df[df['num_paises'] >= 2]
# Adiciona 1 antes do log para lidar com 0 países, evitando log(0)
df['pais_log'] = np.log(df['num_paises'] + 1)

df_nm['pais_log'] = np.log(df_nm['num_paises'])

# Variáveis dependente e de controle

Ao utilizar o max (máximo) para betweenness e degree, o modelo captura o nível mais alto de conectividade direta ou a posição mais forte como intermediário que um autor atingiu em sua trajetória na rede. Isso permite identificar se o potencial máximo de um autor nessas dimensões está associado ao aumento da coautoria internacional, em vez de uma medida diluída pela média ou uma soma total

In [ ]:
def calcular_metricas_max(autores_str, metric_dict):
    if not isinstance(autores_str, str) or autores_str.strip() == '':
        return 0.0

    autores = [a.strip() for a in autores_str.split(';') if a.strip()]
    if not autores:
        return 0.0

    metricas = [metric_dict.get(autor, 0.0) for autor in autores]
    return max(metricas) if metricas else 0.0

# Adicionar colunas de soma de degree e betweenness ao DataFrame
df['degree_max'] = df['Authors'].apply(lambda x: calcular_metricas_max(x, autor_degree))
df['betweenness_max'] = df['Authors'].apply(lambda x: calcular_metricas_max(x, autor_betweenness))

df_nm['degree_max'] = df_nm['Authors'].apply(lambda x: calcular_metricas_max(x, autor_degree))
df_nm['betweenness_max'] = df_nm['Authors'].apply(lambda x: calcular_metricas_max(x, autor_betweenness))

# Regressão Linear

Aqui iremos aplicar uma regressão linear simples em dois casos:
- Modelo Baseline: Entender o impacto de degree na variável dependente
- Modelo completo: Entender o impacto de betweenness na variável dependente

In [ ]:
print("\n--- Realizando Regressão Linear (OLS) ---")
# Variável dependente: log do número de países (pais_log)
# Variáveis preditoras: degree_soma e betweenness_soma

# Definir as variáveis dependente e independentes
y_ols = df_nm['pais_log']
X_ols = df_nm['degree_max']

# Adicionar constante ao modelo
X_ols = sm.add_constant(X_ols)

# Criar e ajustar o modelo OLS
model_ols = sm.OLS(y_ols, X_ols)
results_ols = model_ols.fit()

# Imprimir o resumo dos resultados OLS
print(results_ols.summary())

In [ ]:
import regression as reg
import numpy as np
result = reg.linear(data=df_nm, formula='pais_log ~ degree_max') # dependente à esquerda!
result.micro_summary()

In [ ]:
result.plot_residuals()

# Modelo Baseline

|  |  |
|---|---|
| Variável de controle | Degree |
| Variável dependente | coautoria internacional |
| Expectativa do coeficiente da variável de controle (antes de rodar a regressão) | Positivo |
| Coeficiente da variável de controle (depois de rodar a regressão) | 0.0006 |
| Valor-p da variável de controle (depois de rodar a regressão) | 0 |
| Conclusão sobre a variável de controle | Corroborou (sinal do coeficiente esperado, valor-p baixo) |
| R-quadrado | 0.014 |
| Gráfico de resíduos | ![](graficoresiduo.png) |

In [ ]:
print("\n--- Realizando Regressão Linear (OLS) ---")
# Variável dependente: log do número de países (pais_log)
# Variáveis preditoras: degree_soma e betweenness_soma

# Definir as variáveis dependente e independentes
y_ols = df_nm['pais_log']
X_ols = df_nm[['degree_max', 'betweenness_max']]

# Adicionar constante ao modelo
X_ols = sm.add_constant(X_ols)

# Criar e ajustar o modelo OLS
model_ols = sm.OLS(y_ols, X_ols)
results_ols = model_ols.fit()

# Imprimir o resumo dos resultados OLS
print(results_ols.summary())

In [ ]:
import regression as reg
import numpy as np
result = reg.linear(data=df_nm, formula='pais_log ~ degree_max + betweenness_max') # dependente à esquerda!
result.micro_summary()

In [ ]:
result.plot_residuals()

# Modelo Completo

|  |  |
|---|---|
| Variável de controle | Degree |
| Variável independente | Betweenness centrality |
| Variável dependente | coautoria internacional |
| Expectativa do coeficiente da variável de controle (antes de rodar a regressão) | Positivo |
| Coeficiente da variável de controle (depois de rodar a regressão) | 0.0008 |
| Valor-p da variável de controle (depois de rodar a regressão) | 0 |
| Conclusão sobre a variável de controle | Corroborou (sinal do coeficiente esperado, valor-p baixo) |
| Expectativa do coeficiente da variável independente (antes de rodar a regressão) | Positivo |
| Coeficiente da variável independente (depois de rodar a regressão) | -4.5364 |
| Valor-p da variável independente (depois de rodar a regressão) | 0.118 |
| Conclusão sobre a variável independente | Inverteu (sinal do coeficiente oposto, valor-p baixo) |
| R-quadrado | 0.014 |
| Gráfico de resíduos | ![](graficoresiduo2.png) |

# Novo Contexto

O resultado da regressão, onde a betweenness centrality mostrou um efeito negativo na coautoria internacional (inverso ao esperado), sugere que o mecanismo proposto não se ativa da forma esperada dentro do contexto específico da pesquisa atual. Isso nos leva a investigar as boundary conditions que podem ter interferido.

## Justificativa do resultado

Em um contexto onde as redes de pesquisa internacionais já estão altamente globalizadas e densas, as oportunidades de intermediação por betweenness centrality para "cruzar barreiras" ou "abrir portas" podem ser reduzidas. 

As conexões internacionais já existem de forma mais direta e abundante, e o valor do intermediário (o autor com alta betweenness) diminui porque os pares já estão conectados ou têm múltiplos caminhos para se conectar, sem a necessidade de uma "ponte" específica. O autor com alta betweenness pode ser percebido como um "gargalo" ou um intermediário desnecessário, em vez de um facilitador.

## Proposta de contexto alternativo

Pesquisadores em áreas de pesquisa emergentes ou interdisciplinares que estão em fase inicial de globalização, ou em países em desenvolvimento com um forte ímpeto para internacionalização da ciência, mas com redes internacionais ainda em formação.

## Por que o contexto alternativo é suficientemente SIMILAR ao original?

Os agentes são os mesmos, buscando colaborações. Além disso, o fenômeno de interesse (coautoria internacional) permanece o mesmo, da mesma forma que as métricas de betweenness centrality e degree são relevantes para descrever a estrutura e o posicionamento dos autores dentro da rede. E por fim, permanece a busca por complementaridade metodológica, acesso a dados únicos e negociação de interesses ainda são forças motrizes.

## Por que o contexto alternativo é suficientemente DIFERENTE do original? Em particular, porque ele poderia fazer a conclusao original mudar?

O contexto alternativo difere do original nas condições das "boundary conditions" que deveriam intensificar o efeito da betweenness centrality, em vez de bloqueá-lo ou enfraquecê-lo. O contexto alternativo se concentra em redes que ainda não estão "altamente globalizadas" e onde as conexões internacionais ainda não são abundantes e diretas. Em áreas ou países emergentes na cena global, as lacunas estruturais entre comunidades científicas são mais proeminentes